# 量化交易框架及均线策略演示

本Notebook演示如何使用我们创建的量化交易框架和均线策略模块进行回测。

In [1]:
# 导入必要的库
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 设置中文字体
plt.rcParams['font.family'] = ['SimHei', 'WenQuanYi Micro Hei', 'Heiti TC', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 导入我们创建的量化框架和策略模块
from framework.quant_framework import QuantFramework
from strategies.moving_average_strategy import MovingAverageStrategy, MultipleMovingAverageStrategy

ModuleNotFoundError: No module named 'framework'

## 1. 简单均线交叉策略回测

下面我们将使用简单的均线交叉策略（金叉买入、死叉卖出）进行回测。

In [ ]:
# 初始化量化框架
framework = QuantFramework()

# 设置回测参数
symbol = 'CZCE.FG601'  # 玻璃期货合约
start_date = date(2023, 1, 1)  # 回测开始日期
end_date = date(2023, 12, 31)  # 回测结束日期
initial_capital = 100000  # 初始资金

# 初始化框架
# 注意：如需使用实盘或完整回测功能，需要填写真实的天勤账户和密码
# framework.initialize(symbol, start_date, end_date, initial_capital, '您的天勤账户', '您的天勤密码')
framework.initialize(symbol, start_date, end_date, initial_capital)

# 创建均线策略实例（短周期5，长周期20）
ma_strategy = MovingAverageStrategy(short_period=5, long_period=20)

# 设置策略
framework.set_strategy(ma_strategy)

# 运行回测
# framework.run_backtest()  # 取消注释以运行回测

## 2. 多均线策略回测

下面我们将使用多均线策略（基于三条均线的排列关系）进行回测。

In [ ]:
# 初始化新的量化框架实例
multi_framework = QuantFramework()

# 初始化框架（使用相同的回测参数）
multi_framework.initialize(symbol, start_date, end_date, initial_capital)

# 创建多均线策略实例（短周期5，中周期10，长周期20）
multi_ma_strategy = MultipleMovingAverageStrategy(short_period=5, mid_period=10, long_period=20)

# 设置策略
multi_framework.set_strategy(multi_ma_strategy)

# 运行回测
# multi_framework.run_backtest()  # 取消注释以运行回测

## 3. 不同参数的均线策略对比

下面我们将演示如何对比不同参数设置下的均线策略表现。

In [ ]:
def compare_strategy_parameters(symbol, start_date, end_date, param_combinations):
    '''
    对比不同参数组合的策略表现
    
    Args:
        symbol: 交易品种代码
        start_date: 回测开始日期
        end_date: 回测结束日期
        param_combinations: 参数组合列表
    '''
    results = []
    
    for params in param_combinations:
        short_period, long_period = params
        print('
测试参数组合: 短周期={}, 长周期={}'.format(short_period, long_period))
        
        # 初始化框架
        framework = QuantFramework()
        framework.initialize(symbol, start_date, end_date, initial_capital)
        
        # 创建策略
        strategy = MovingAverageStrategy(short_period=short_period, long_period=long_period)
        framework.set_strategy(strategy)
        
        # 运行回测（实际运行时取消注释）
        # framework.run_backtest()
        
        # 这里只是模拟结果，实际运行时会从回测结果中获取
        # 注意：在实际运行时，应该记录真实的回测结果
        result = {
            'short_period': short_period,
            'long_period': long_period,
            'total_return': np.random.uniform(-0.1, 0.3),  # 模拟总收益率
            'max_drawdown': np.random.uniform(0.05, 0.2),  # 模拟最大回撤
            'trade_count': np.random.randint(10, 50)  # 模拟交易次数
        }
        results.append(result)
    
    return pd.DataFrame(results)

# 定义要测试的参数组合
param_combinations = [(5, 20), (10, 30), (5, 60), (15, 60)]

# 运行参数对比（实际运行时取消注释）
# results_df = compare_strategy_parameters(symbol, start_date, end_date, param_combinations)
# print(results_df)

## 4. 自定义策略开发

下面我们将演示如何基于我们的框架开发自定义策略。

In [ ]:
from quant_framework import StrategyBase
from tqsdk import TargetPosTask
from tqsdk.ta import MA, STD

class BollingerBandStrategy(StrategyBase):
    '''
    布林带策略
    基于价格突破布林带上轨和下轨进行交易
    '''
    def __init__(self, period=20, k=2, kline_period=60*60*24):
        '''
        初始化布林带策略
        
        Args:
            period: 计算布林带的周期，默认为20
            k: 标准差倍数，默认为2
            kline_period: K线周期，单位为秒，默认为日线
        '''
        super().__init__()
        self.period = period
        self.k = k
        self.kline_period = kline_period
        self.klines = None
        self.mid_band = None
        self.upper_band = None
        self.lower_band = None
        self.target_pos = None
        self.position = 0
        
    def initialize(self, api, symbol):
        '''
        初始化策略
        
        Args:
            api: TqApi实例
            symbol: 交易品种代码
        '''
        super().initialize(api, symbol)
        
        # 获取K线数据
        self.klines = api.get_kline_serial(symbol, self.kline_period)
        
        # 创建TargetPosTask用于自动调整持仓
        self.target_pos = TargetPosTask(api, symbol)
        
        # 打印策略参数
        print('布林带策略参数: 周期={}, 标准差倍数={}'.format(self.period, self.k))
        
    def run(self):
        '''
        运行策略
        '''
        while True:
            # 等待K线更新
            self.api.wait_update()
            
            # 如果K线最后一根发生变化
            if self.api.is_changing(self.klines.iloc[-1], 'datetime'):
                # 确保有足够的数据计算指标
                if len(self.klines) >= self.period:
                    # 计算布林带
                    self._calculate_bollinger_bands()
                    
                    # 生成交易信号
                    self._generate_signals()
                    
                # 更新性能指标
                self.update_performance()
    
    def _calculate_bollinger_bands(self):
        '''
        计算布林带指标
        '''
        # 计算均线（中轨）
        self.mid_band = MA(self.klines, self.period)
        
        # 计算标准差
        std = STD(self.klines, self.period)
        
        # 计算上轨和下轨
        self.upper_band = self.mid_band + self.k * std
        self.lower_band = self.mid_band - self.k * std
    
    def _generate_signals(self):
        '''
        生成交易信号
        '''
        # 确保数据有效
        if len(self.mid_band) < 1 or len(self.upper_band) < 1 or len(self.lower_band) < 1:
            return
        
        try:
            # 获取当前价格和布林带值
            current_price = self.klines.iloc[-1].close
            upper_band_value = self.upper_band.iloc[-1]
            lower_band_value = self.lower_band.iloc[-1]
            
            # 价格跌破下轨，买入信号
            if current_price < lower_band_value and self.position <= 0:
                print('价格跌破下轨: 买入 {}, 价格: {:.2f}'.format(self.symbol, current_price))
                self.target_pos.set_target_volume(1)
                self.position = 1
                self.trade_count += 1
            
            # 价格突破上轨，卖出信号
            elif current_price > upper_band_value and self.position >= 0:
                print('价格突破上轨: 卖出 {}, 价格: {:.2f}'.format(self.symbol, current_price))
                self.target_pos.set_target_volume(-1)
                self.position = -1
                self.trade_count += 1
        except IndexError:
            return

# 创建并测试自定义策略（实际运行时取消注释）
# custom_framework = QuantFramework()
# custom_framework.initialize(symbol, start_date, end_date, initial_capital)
# bollinger_strategy = BollingerBandStrategy(period=20, k=2)
# custom_framework.set_strategy(bollinger_strategy)
# custom_framework.run_backtest()

## 5. 总结

通过本Notebook，我们演示了如何使用我们创建的量化交易框架和均线策略模块进行回测。主要功能包括：

1. 简单均线交叉策略的使用
2. 多均线策略的使用
3. 不同参数组合的策略对比
4. 基于框架开发自定义策略

要实际运行这些策略，您需要：
1. 注册天勤量化账户（https://account.shinnytech.com/）
2. 在代码中填写您的天勤账户和密码
3. 取消相应代码行的注释并运行

请注意，回测结果并不能保证未来的收益，实盘交易存在风险，请谨慎决策。